In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2

In [ ]:
from IPython.core.debugger import set_trace

In [ ]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.models import load_model


multi_model = load_model('models/multi_model__sweep=15_decimation=2_numclasses=3_valloss=0.057.h5')
print(multi_model.inputs)

for l in multi_model.layers:
    print('{}: {}'.format(l.name, 'trainable' if l.trainable else 'not-trainable'))

SVG(model_to_dot(multi_model).create(prog='dot', format='svg'))

In [ ]:
import pickle
import time
import numpy as np

import keras.backend as K
from keras.models import Model
from keras.layers import Add, Subtract, Average, Flatten, Reshape, Dense, Input, Lambda, Concatenate, Softmax, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, UpSampling2D
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.callbacks import EarlyStopping
from keras.layers import ConvLSTM2D, TimeDistributed

from utils import (
    sequential_batcher, extract_observation_for_batch,
    get_X_and_Y, plot_semantic
)


def get_recur_model(multi_model, seq_len, input_names, train_encoder=True, train_decoder=True):
    input_shape =  K.int_shape(multi_model.input[0])[1:]
    seq_input_shape = (seq_len, *input_shape)
    encoded_dim = K.int_shape(multi_model.get_layer('encoder_submodel').outputs[0])[1:]

    inputs = {
        inp_name: Input(seq_input_shape, name='input_from_'+inp_name)
        for inp_name in input_names
    }

    decoder = multi_model.get_layer('decoder_submodel')
    decoder.trainable = train_decoder
    
    previous_reconstruction_model = Model(
        [multi_model.inputs[inp_idx] for inp_idx, inp_name in enumerate(input_names)],
        multi_model.get_layer('before_reconstruction').output,
    )

    for_reconstruction = []
    for inp_idx, inp_name in enumerate(input_names):
        encoder = Model(
            multi_model.inputs[inp_idx],
            multi_model.get_layer('encoded_from_'+inp_name).output,
        )
        encoder.trainable = train_encoder

        inp = inputs[inp_name]
        x = TimeDistributed(encoder)(inp)
        for_reconstruction.append(x)

    x = Concatenate()(for_reconstruction)
    rec = ConvLSTM2D(
            encoded_dim[-1],
            (3, 3),
            padding='same',
            return_sequences=False,
    )(x)
    rec = Convolution2D(
        encoded_dim[-1],
        (3, 3),
        activation='elu',
        padding='same',
    )(rec)
    rec = decoder(rec)
    
    previous_reconstruction = previous_reconstruction_model([
        Lambda(
            lambda x: x[:, -1, :, :, :],
        )(inputs[inp_name])
        for inp_name in input_names
    ])
    previous_reconstruction = decoder(previous_reconstruction)
    
    added_reconstructions = Add()([rec, previous_reconstruction])
    out = Softmax(axis=3)(added_reconstructions)

    return Model([inputs[inp_name] for inp_name in input_names], out)

In [ ]:
DECIMATION = 2
SEQUENCE_LEN = 4
SEQ_BATCH_SIZE = 8
CLASSES_NAMES = [
    ['Roads', 'RoadLines'],
    
    ['None', 'Buildings', 'Fences', 'Other', 'Pedestrians',
     'Poles', 'Walls', 'TrafficSigns',
     'Vegetation', 'Sidewalks'],
    
    ['Vehicles'],
]

# CLASSES_NAMES = [
#     ['Roads', 'RoadLines'],
    
#     ['Sidewalks'],
    
#     ['Buildings'],
    
#     ['Fences', 'Other', 'Pedestrians',
#      'Poles', 'Walls', 'TrafficSigns'],
    
#     ['Vehicles'],
    
#     ['Vegetation'],
        
#     ['None']
# ]


patience = 10


input_names = ['FrontSS', 'LeftSS', 'RightSS', 'RearSS']
train_encoder = False
train_decoder = False


recur_model = get_recur_model(
    multi_model,
    SEQUENCE_LEN,
    input_names,
    train_encoder,
    train_decoder,
)
recur_model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(1e-5),  # needs to be low!!!!
)

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=patience,
    restore_best_weights=True,
)

X_val, Y_val = get_X_and_Y(['Town01', 'Town02'], [100, 101], DECIMATION)
valid_seq_gen = sequential_batcher(
    X_val, Y_val,
    return_sequences=False,
    batch_size=SEQ_BATCH_SIZE,
    sequence_len=SEQUENCE_LEN,
    classes_names=CLASSES_NAMES,
    val_part=1.0,
    validation=True,
)

In [ ]:
RECURRENT_EPISODES = [
    range(20, 27),
    range(27, 34),
    range(34, 40),
]


for sweep in range(12):
    histories = []
    for episodes in RECURRENT_EPISODES:
        start_time = time.time()
        X, Y = get_X_and_Y(['Town01', 'Town02'], episodes, DECIMATION)
        print('Reading data took {:.2f} [s]'.format(time.time() - start_time))

        train_seq_gen = sequential_batcher(
            X, Y,
            return_sequences=False,
            batch_size=SEQ_BATCH_SIZE,
            sequence_len=SEQUENCE_LEN,
            classes_names=CLASSES_NAMES,
            val_part=0.0,
            validation=False,
        )
        
        one_batch_X, one_batch_Y = next(train_seq_gen)

        preds = recur_model.predict(one_batch_X)

        sep = np.zeros_like(preds[0:1, :, ::5])
        for j in range(len(preds)):
            plot_semantic(np.concatenate([one_batch_Y[j:j+1], sep, preds[j:j+1]], axis=2))
        
        print('\n#### episodes = {} #### (sweep: {})'.format(episodes, sweep))

        history = recur_model.fit_generator(
            train_seq_gen,
            steps_per_epoch=X[0].shape[-1] // SEQ_BATCH_SIZE // 10,
            epochs=50,
            validation_data=valid_seq_gen,
            validation_steps=X_val[0].shape[-1] // SEQ_BATCH_SIZE // 2,
            callbacks=[early_stopping],
        )
        
        histories.append(history.history)

        del X
        del Y
        
    val_loss = history.history['val_loss'][-(patience+1)]
    model_filename = 'models/recur_model__sweep={}_decimation={}_numclasses={}_valloss={:.3f}.h5'.format(sweep, DECIMATION, len(CLASSES_NAMES), val_loss)
    multi_model.save(model_filename)
    histories_filename = 'histories/recur_model__sweep={}_decimation={}_numclasses={}_valloss={:.3f}.pkl'.format(sweep, DECIMATION, len(CLASSES_NAMES), val_loss)
    with open(histories_filename, 'wb') as output:
        pickle.dump(histories, output)